In [ ]:
!pip install pygame

In [ ]:
!pip install -q -U kaggle_environments

In [ ]:
import json
import random
from kaggle_environments import make

# Load keywords from the provided file
with open('/kaggle/input/llm-20-questions/llm_20_questions/keywords.py') as file:
    file_content = file.read()

# Extract JSON part
start_index = file_content.find('[')
end_index = file_content.rfind(']') + 1
keywords_json_str = file_content[start_index:end_index]

# Parse JSON
keywords = json.loads(keywords_json_str)

# Organize keywords by category
keyword_dict = {}
for category_data in keywords:
    category = category_data["category"]
    words = category_data["words"]
    keyword_dict[category] = {word["keyword"]: word["alts"] for word in words}

# Example keyword the answerer is thinking of
answerer_category = random.choice(list(keyword_dict.keys()))
answerer_word = random.choice(list(keyword_dict[answerer_category].keys()))

# Memory structure to track interactions
guesser_memory = {
    "questions": [],
    "answers": [],
    "categories": []
}

# Metrics tracking
metrics = {
    "total_reward": 0,
    "episodes": 0,
    "successes": 0
}

# Agent logic for the guesser
def guesser_agent(observation, configuration):
    if observation["turnType"] == "ask":
        # Choose a category not yet confirmed
        possible_categories = list(keyword_dict.keys())
        if guesser_memory["categories"]:
            possible_categories = [cat for cat in possible_categories if cat not in guesser_memory["categories"]]

        if not possible_categories:
            possible_categories = list(keyword_dict.keys())

        category = random.choice(possible_categories)
        question = f"Is it related to {category}?"
        guesser_memory["questions"].append(question)
        return question

    elif observation["turnType"] == "guess":
        # Make a guess based on the confirmed category
        confirmed_category = None
        if "yes" in guesser_memory["answers"]:
            confirmed_category = guesser_memory["categories"][guesser_memory["answers"].index("yes")]
        if not confirmed_category:
            confirmed_category = random.choice(list(keyword_dict.keys()))

        guess = random.choice(list(keyword_dict[confirmed_category].keys()))
        guesser_memory["questions"].append(guess)
        return guess

# Agent logic for the answerer
def answerer_agent(observation, configuration):
    if observation["turnType"] == "answer":
        question = observation["questions"][-1]
        category = question.split()[-1][:-1]  # Extract category from question
        guesser_memory["categories"].append(category)
        if category == answerer_category:
            answer = "yes"
        else:
            answer = "no"
        guesser_memory["answers"].append(answer)
        return answer

# Define the agent function required by the environment
def agent(observation, configuration):
    if observation["role"] == "guesser":
        return guesser_agent(observation, configuration)
    else:
        return answerer_agent(observation, configuration)

# Create the environment
env = make("llm_20_questions", debug=True)

# Run multiple episodes to gather metrics
num_episodes = 100
for episode in range(num_episodes):
    env.reset()
    steps = env.run([agent, agent, agent, agent])
    
    # Calculate rewards and success
    reward = sum(step[0]['reward'] for step in steps)
    metrics["total_reward"] += reward
    metrics["episodes"] += 1
    if steps[-1][0]['status'] == 'SUCCESS':
        metrics["successes"] += 1

# Calculate average reward per episode
metrics["average_reward"] = metrics["total_reward"] / metrics["episodes"]
metrics["success_rate"] = metrics["successes"] / metrics["episodes"]

print("Metrics:", metrics)

# Render the last episode to visualize the game
env.render(mode="ipython", width=800, height=700)

# Submit File To Competition

In [ ]:
# Create a Python script named submission.py with the provided code
script_content = """
import json
import random
from kaggle_environments import make

# Load keywords from the provided file
with open('/kaggle/input/llm-20-questions/llm_20_questions/keywords.py') as file:
    file_content = file.read()

# Extract JSON part
start_index = file_content.find('[')
end_index = file_content.rfind(']') + 1
keywords_json_str = file_content[start_index:end_index]

# Parse JSON
keywords = json.loads(keywords_json_str)

# Organize keywords by category
keyword_dict = {}
for category_data in keywords:
    category = category_data["category"]
    words = category_data["words"]
    keyword_dict[category] = {word["keyword"]: word["alts"] for word in words}

# Example keyword the answerer is thinking of
answerer_category = random.choice(list(keyword_dict.keys()))
answerer_word = random.choice(list(keyword_dict[answerer_category].keys()))

# Memory structure to track interactions
guesser_memory = {
    "questions": [],
    "answers": [],
    "categories": []
}

# Metrics tracking
metrics = {
    "total_reward": 0,
    "episodes": 0,
    "successes": 0
}

# Agent logic for the guesser
def guesser_agent(observation, configuration):
    if observation["turnType"] == "ask":
        # Choose a category not yet confirmed
        possible_categories = list(keyword_dict.keys())
        if guesser_memory["categories"]:
            possible_categories = [cat for cat in possible_categories if cat not in guesser_memory["categories"]]

        if not possible_categories:
            possible_categories = list(keyword_dict.keys())

        category = random.choice(possible_categories)
        question = f"Is it related to {category}?"
        guesser_memory["questions"].append(question)
        return question

    elif observation["turnType"] == "guess":
        # Make a guess based on the confirmed category
        confirmed_category = None
        if "yes" in guesser_memory["answers"]:
            confirmed_category = guesser_memory["categories"][guesser_memory["answers"].index("yes")]
        if not confirmed_category:
            confirmed_category = random.choice(list(keyword_dict.keys()))

        guess = random.choice(list(keyword_dict[confirmed_category].keys()))
        guesser_memory["questions"].append(guess)
        return guess

# Agent logic for the answerer
def answerer_agent(observation, configuration):
    if observation["turnType"] == "answer":
        question = observation["questions"][-1]
        category = question.split()[-1][:-1]  # Extract category from question
        guesser_memory["categories"].append(category)
        if category == answerer_category:
            answer = "yes"
        else:
            answer = "no"
        guesser_memory["answers"].append(answer)
        return answer

# Define the agent function required by the environment
def agent(observation, configuration):
    if observation["role"] == "guesser":
        return guesser_agent(observation, configuration)
    else:
        return answerer_agent(observation, configuration)

# Create the environment
env = make("llm_20_questions", debug=True)

# Run multiple episodes to gather metrics
num_episodes = 100
for episode in range(num_episodes):
    env.reset()
    steps = env.run([agent, agent, agent, agent])
    
    # Calculate rewards and success
    reward = sum(step[0]['reward'] for step in steps)
    metrics["total_reward"] += reward
    metrics["episodes"] += 1
    if steps[-1][0]['status'] == 'SUCCESS':
        metrics["successes"] += 1

# Calculate average reward per episode
metrics["average_reward"] = metrics["total_reward"] / metrics["episodes"]
metrics["success_rate"] = metrics["successes"] / metrics["episodes"]

print("Metrics:", metrics)

# Render the last episode to visualize the game
env.render(mode="ipython", width=800, height=700)
"""

# Write the content to submission.py
submission_file_path = '/kaggle/working/submission.py'
with open(submission_file_path, 'w') as file:
    file.write(script_content)

submission_file_path